# LeetCode 1765
![lc-1765](./assets/question.jpg)
![lc-1765](./assets/constraints.jpg)

> Observations:
> - For all given matrices, there is always AT LESAT ONE water cell
> - For isWater, 1 = water level and 0 = land cell (with to-be defined level)
> - Any adjacent cells must have an absolute height difference of at most 1
> - Note that matrix is of m * n, not n * n

![lc-1765-ex1](./assets/ex1.jpg)
![lc-1765-ex2](./assets/ex2.jpg)

> Notes:
> - All cells next to water cells are at an elevation level of one unit, while adjacent cells follow the rule of being at most one level of elevation higher
> - With the note above, this question suggest the use of a breadth first search (BFS) solution implementation where we first find all locations of sea-level cells (water cells)
> - Then, with inspiration from the A-star pathfinder algorithm, we move in the cardinal directions to set the sea level elevations on the matrix
> - But of course, we do not want to overwrite any of the cells we have already visited, so we need a list to keep track of all cell locations that have been visited